In [ ]:
! pip install datasets transformers

In [ ]:
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
from datasets import load_dataset, load_metric
import os
# datasets = load_dataset("squad_v2" if squad_v2 else "squad")
# datasets
datasets = load_dataset('json', data_files={'train': os.listdir('csv_renamed'),
                                              'test': '/.../js1.json'}, field='data')
datasets

Using custom data configuration default-67c0e7d1e43dac1a


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-67c0e7d1e43dac1a/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'id', 'title', 'answers'],
        num_rows: 2160
    })
    test: Dataset({
        features: ['context', 'question', 'id', 'title', 'answers'],
        num_rows: 1080
    })
})

In [ ]:
datasets["train"][0]

{'answers': {'answer_start': [], 'text': []},
 'context': 'Molecular Genetics and Metabolism 121 (2017) 1–8\\n',
 'id': '0_0_First Author',
 'question': 'First Author',
 'title': '00000_1-s2.0-S1096719217300525-main.txt'}

In [ ]:
x = datasets["train"][0]

x['context'][515:550]

''

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))
show_random_elements(datasets["train"], num_examples=2)

,context,question,id,title,answers
0,"Adults with phenylketonuria (PKU) may experience neurologic and psychiatric disorders, including intellectual\ndisability, anxiety, depression, and neurocognitive dysfunction. Identifying the prevalence and prevalence ratios\nof these conditions will inform clinical treatment. This nested, case-controlled study used International Classiﬁ-\ncation of Diseases, Ninth Revision (ICD-9) codes from the MarketScan® insurance claims databases from 2006 to\n2012 and healthcare claims data for US-based employer and government-sponsored health plans. Prevalence and\nprevalence ratio calculations of neuropsychiatric comorbidities for adults (≥20 years old) with PKU were com-\npared with two groups [diabetes mellitus (DM) and general population (GP)] matched by age, gender, geograph-\nic location, and insurance type. Age cohorts (i.e., 20–29, 30–39, 40–49, 50–59, 60–69, and 70 + years, and a\ncombined subset of 20–39) were used to stratify data. The PKU cohort experienced signiﬁcantly higher rates of\nseveral comorbid neurologic, psychiatric and developmental conditions. Compared to GP, PKU was associated\nwith signiﬁcantly higher prevalence for numerous neuropsychiatric conditions, most notably for intellectual dis-\nability (PR = 7.9, 95% CI: 6.4–9.9), autism spectrum disorder (PR = 6.1, 95% CI: 3.6–10.4), Tourette/tic disorders\n(PR = 5.4, 95% CI: 2.1–14.1), and eating disorders (4.0, 95% CI: 3.2–5.0). Rates of fatigue/malaise, epilepsy/con-\nvulsions, sleep disturbance, personality disorders, phobias, psychosis, and migraines among those with PKU\nexceeded rates for the GP but were comparable to those with DM, with signiﬁcantly lower rates of concomitant\ndisorders occurring in younger, compared to older, adults with PKU. Lifelong monitoring and treatment of co-oc-\ncurring neuropsychiatric conditions are important for effective PKU management.\n",Study Title,0_11_Study Title,00000_1-s2.0-S1096719217300525-main.txt,"{'answer_start': [], 'text': []}"
1,"[49] M. Bik-Multanowski, J.J. Pietrzyk, R. Mozrzymas, Routine use of CANTAB system for\ndetection of neuropsychological deﬁcits in patients with PKU, Mol. Genet. Metab.\n102 (2) (2011) 210–213, http://dx.doi.org/10.1016/j.ymgme.2010.10.003.\n",Study Description,0_250_Study Description,00000_1-s2.0-S1096719217300525-main.txt,"{'answer_start': [], 'text': []}"


In [ ]:
from transformers import AutoTokenizer
import transformers

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

528

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)
[len(x) for x in tokenized_example["input_ids"]]

[384, 277]

In [ ]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    print(examples)
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


# model(prepare_train_features(datasets["train"][0]))
pad_on_right = tokenizer.padding_side == "right"
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

{'context': ['Molecular Genetics and Metabolism 121 (2017) 1–8\\n', 'Molecular Genetics and Metabolism 121 (2017) 1–8\\n', 'Molecular Genetics and Metabolism 121 (2017) 1–8\\n', 'Molecular Genetics and Metabolism 121 (2017) 1–8\\n', 'Contents lists available at ScienceDirect\\n', 'Contents lists available at ScienceDirect\\n', 'Contents lists available at ScienceDirect\\n', 'Contents lists available at ScienceDirect\\n', 'Molecular Genetics and Metabolism\\n', 'Molecular Genetics and Metabolism\\n', 'Molecular Genetics and Metabolism\\n', 'Molecular Genetics and Metabolism\\n', 'j o u r n a l h o m e p a g e : w w w . e l s e v i e r . c o m / l o c a t e / y m g m e\\n', 'j o u r n a l h o m e p a g e : w w w . e l s e v i e r . c o m / l o c a t e / y m g m e\\n', 'j o u r n a l h o m e p a g e : w w w . e l s e v i e r . c o m / l o c a t e / y m g m e\\n', 'j o u r n a l h o m e p a g e : w w w . e l s e v i e r . c o m / l o c a t e / y m g m e\\n', 'Neuropsychiatric comorbidities

{'answers': [{'answer_start': [], 'text': []}, {'answer_start': [39], 'text': ['2017']}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text': []}, {'answer_start': [], 'text'

In [ ]:
print(type(tokenized_datasets), len(tokenized_datasets))
# print(tokenized_datasets.

<class 'datasets.dataset_dict.DatasetDict'> 2


In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
datasets["train"][0]

{'answers': {'answer_start': [], 'text': []},
 'context': 'Molecular Genetics and Metabolism 121 (2017) 1–8\\n',
 'id': '0_0_First Author',
 'question': 'First Author',
 'title': '00000_1-s2.0-S1096719217300525-main.txt'}

In [ ]:
args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=300,
    weight_decay=0.01,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,0.011106,25.400000,43.150000
2,No log,0.001018,25.374300,43.193000
3,No log,0.000299,25.384600,43.176000
4,0.165500,0.000237,25.386600,43.172000
5,0.165500,0.000183,25.363000,43.212000
6,0.165500,0.000159,25.358600,43.220000
7,0.165500,0.000139,25.313000,43.298000
8,0.002300,0.000351,25.415300,43.124000
9,0.002300,0.000099,25.404300,43.142000
10,0.002300,0.000089,25.360300,43.217000
